## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Busselton,AU,-33.6500,115.3333,42.04,83,7,7.94,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,75.74,77,100,18.41,overcast clouds
2,2,Hermanus,ZA,-34.4187,19.2345,61.90,89,9,5.12,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,49.62,46,20,25.32,few clouds
4,4,Cidreira,BR,-30.1811,-50.2056,68.99,83,75,21.07,broken clouds


In [3]:
# Input statements
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Rikitea,PF,-23.1203,-134.9692,75.74,77,100,18.41,overcast clouds
5,5,Cairns,AU,-16.9167,145.7667,77.07,85,40,5.75,scattered clouds
14,14,Boueni,YT,-12.9025,45.0761,76.98,69,23,5.97,few clouds
19,19,Bandarbeyla,SO,9.4942,50.8122,78.15,82,11,14.65,few clouds
20,20,Sur,OM,22.5667,59.5289,77.81,49,1,1.43,clear sky
26,26,Atuona,PF,-9.8000,-139.0333,77.88,72,14,13.58,few clouds
31,31,Saint George,US,37.1041,-113.5841,78.53,31,1,1.99,clear sky
38,38,Hilo,US,19.7297,-155.0900,80.26,68,40,8.05,scattered clouds
40,40,Faanui,PF,-16.4833,-151.7500,78.73,70,100,15.14,overcast clouds
42,42,Ca Mau,VN,9.1769,105.1500,76.35,94,99,4.23,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                213
City                   213
Country                212
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Weather Description    213
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any',axis=0)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.74,overcast clouds,-23.1203,-134.9692,
5,Cairns,AU,77.07,scattered clouds,-16.9167,145.7667,
14,Boueni,YT,76.98,few clouds,-12.9025,45.0761,
19,Bandarbeyla,SO,78.15,few clouds,9.4942,50.8122,
20,Sur,OM,77.81,clear sky,22.5667,59.5289,
26,Atuona,PF,77.88,few clouds,-9.8000,-139.0333,
31,Saint George,US,78.53,clear sky,37.1041,-113.5841,
38,Hilo,US,80.26,scattered clouds,19.7297,-155.0900,
40,Faanui,PF,78.73,overcast clouds,-16.4833,-151.7500,
42,Ca Mau,VN,76.35,moderate rain,9.1769,105.1500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   212
Country                212
Max Temp               212
Weather Description    212
Lat                    212
Lng                    212
Hotel Name             212
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))